# Adnotacja danych

Adnotacja danych jest jednym z najważniejszych procesów przy trenowaniu statystycznych modeli uczących pracujących na tekście. Do adnotacji można wykorzystać zarówno narzędzia open source, takie jak [Doccano](https://github.com/doccano/doccano), narzędzia komercyjne pokroju [Prodigy](http://prodi.gy), jak i narzędzia do adnotacji programistycznej, choćby znakomity [Snorkel](https://www.snorkel.org).

W tym ćwiczeniu zobaczymy dwa podejścia. W pierwszym kroku wykorzystamy wbudowany w bibliotekę `SpaCy` mechanizm regułowego oznaczania danych. W drugim kroku wykorzystamy Prodigy.

## Adnotacja regułowa z użyciem SpaCy

In [ ]:
import spacy

nlp = spacy.load('pl_core_news_md')

In [ ]:
with open('pantadeusz.txt','rt') as f:
    text = f.read().replace('\n', ' ')
    
text[:1000]

In [ ]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)

pattern = [{"LOWER": "pan"}, {"POS": "NOUN"}]
matcher.add("PanJakiśTam", None, pattern)

doc = nlp(text)
matches = matcher(doc)

for match_id, start, end in matches:
    span = doc[start:end]
    
    print(start, end, span.text)

In [ ]:
matcher = Matcher(nlp.vocab)

patterns = [
    [{'POS': 'ADJ'}, {'ENT_TYPE': 'placeName'}],
    [{'POS': 'ADJ'}, {'ENT_TYPE': 'persName'}],
]
matcher.add(None, None, *patterns)

doc = nlp(text)
matches = matcher(doc)

for match_id, start, end in matches:
    span = doc[start:end]
    
    print(match_id, start, end, span.text)

In [ ]:
ent_types = []

for e in doc.ents:
    ent_types.append(e.label_)
    
print(set(ent_types))

### zadanie samodzielne

Przygotuj *matcher* który znajdzie w treści "*Pana Tadeusza*" wszystkie rzeczowniki występujące bezpośrednio przed lub po jakiejś dacie rocznej.

In [ ]:
matcher = Matcher(nlp.vocab)

patterns = [
    # uzupełnij definicję wzorców
]
matcher.add("DatyRoczne", None, *patterns)

doc = nlp(text)
matches = matcher(doc)

for match_id, start, end in matches:
    span = doc[start:end]
    
    print(match_id, start, end, span.text)

## Adnotacja przy użyciu Prodigy

1. Utworzenie zbioru danych

```bash
prodigy dataset owoce "Zbiór słów które posłużą jako przykładowe owoce"
```

2. Wykorzystanie przestrzeni wektorowej do rozszerzenia zbioru słów reprezentujących owoce

```bash
prodigy terms.teach owoce pl_core_news_lg --seeds jabłko,gruszka,wiśnia,porzeczki
```

3. Zapisanie owoców w postaci wzorców dostępnych dla uczenia modelu

```bash
prodigy terms.to-patterns owoce owoce.jsonl --spacy-model pl_core_news_lg --label OWOC
```

4. Utworzenie zbioru uczącego 

```bash
prodigy dataset owoce_ner "Adnotacje z lokalizacją owoców w Panu Tadeuszu"
```

5. Ręczna adnotacja danych z użyciem modelu 

```bash
prodigy ner.teach owoce_ner pl_core_news_lg pantadeusz.txt --loader txt 
    --label OWOC --patterns owoce.jsonl --unsegmented
```

6. Wytreniowanie modelu rozpoznawania owoców

```bash
prodigy train ner owoce_ner pl_core_news_lg --output /tmp/model --n-iter 100
```

In [ ]:
import spacy

nlp = spacy.load('/tmp/model')

phrases = [
    'W tej części sadu rosły tu i ówdzie wiśnie, ',
    'Zlany granatem czarnej, zgniecionej jagody ',
    'Nieszczęściem, trącił krzaki suchego agrestu. '
]

for doc in nlp.pipe(phrases):
    print(doc)
    for e in doc.ents:
        print(e.label_, doc[e.start:e.end])